In [10]:
import time
import torch

# Scaling Up Forecast Horizon

Most deep forecasting models work as a **settled function** once trained. For scenarios where the prediction horizon is mismatched or long-term. The most common practice is rolling forecast.

But one of the essential characteristics of non-stationary data is that the distribution changes significantly over time. Thus the forecasting error can be large at incoming time points, otherwise the model should be continously retrained. 

Therefore, it poses two challenges for current model application pipeline: 
- (1) reuse parameters learned from observed series; 
- (2) utilize incoming ground truth for model adaptation. 
 
The practical scenarios, which we name as scaling up forecast horizon, may lead to failure on most deep models but can be naturally tackled by Koopa.

<p align="center">
<img src="./figures/adapt_demo.png" height = "150" alt="" align=center />
</p>

## Operator Adaptation

In detail, we train a Koopa model with an initial forecast length and attempt to apply it on a larger one. 

The basic approach conducts rolling forecast by taking the model prediction as the input of the next iteration until the desired forecast horizon is all filled.

Instead, we assume (commonly in real situations) that after the model gives a prediction, the model can utilize the incoming ground truth for **operator adaptation** and continue rolling forecast for the next iteration. 

In [11]:
def operator_adaptation(Z, incoming_list):
    B, G, D = incoming_list.shape
    Zp, Zf = Z[:, :-1], Z[:, 1:]
    # the same as torch.linalg.pinv(Zp) @ Zf
    K = torch.linalg.lstsq(Zp, Zf).solution 
    n, pred_list = Z[:, -1:], [Z[:, -1:] @ K]
    # K_list = [K]

    for i in range(G):
        m, n = n, incoming_list[:, i].unsqueeze(1)
        Zp = torch.cat((Zp, m), dim=1)
        Zf = torch.cat((Zf, n), dim=1)
        K = torch.linalg.lstsq(Zp, Zf).solution
        pred_list.append(n @ K)
        # K_list.append(K)
        
    return torch.concat(pred_list, dim=1) # ,torch.stack(K_list, dim=0)

The naive implementation shown as the above repeatedly conducts DMD on the incremental embedding collection to obtain new operators.

In [12]:
B = 64       # Batch size
F = 10       # Number of observed snappshots
D = 1024     # Dimension of Koopman embedding
G = 10       # Number of successively incoming snapshots

Z = torch.randn(B, F, D)                # Observed time points
incoming_list = torch.randn(B, G, D)    # Successively incoming time points

pred = operator_adaptation(Z, incoming_list) # Predicted time points (unseen the previous groundtruth)
print(pred.shape)

torch.Size([64, 11, 1024])


 It is notable that we do not retrain parameters during model adaptation, since it will lead to overfitting on the incoming ground truth and **Catastrophic Forgetting**.

This algorithm (Koopa OA) can make the model adapt to the change of data distribution without training, and further improve the prediction effect, especially in the **non-stationary time series**.

<p align="center">
<img src="./figures/adaptation.png" alt="" align=center />
</p>

## Computational Acceleration

The naive implementation has a complexity of $\mathcal{O}(LD^3)$, where $L$ is the number of incoming time points and $D$ is the dimension of Koopman embedding. Based on the linearity of Koopman operators, we propose an **equivalent algorithm with improved complexity** of $\mathcal{O}((L+D)D^2)$.

In [13]:
def operator_adaptation_accelerate(Z, incoming_list):
    B, G, D = incoming_list.shape
    Zp, Zf = Z[:, :-1], Z[:, 1:]
    Zp_inv = torch.linalg.pinv(Zp)
    K = Zp_inv @ Zf
    X = Zp_inv @ Zp
    n, pred_list = Z[:, -1:], [Z[:, -1:] @ K]
    # K_list = [K]

    for i in range(G):
        m, n = n, incoming_list[:, i].unsqueeze(1)
        mt = m.transpose(1, 2)
        r = mt - X.transpose(1, 2) @ mt
        b = r / r.square().sum(dim=1, keepdim=True)
        K = K - b @ (m @ K - n)
        X = X - b @ (m @ X - m)
        pred_list.append(n @ K)
        # K_list.append(K)
        
    return torch.concat(pred_list, dim=1) # ,torch.stack(K_list, dim=0)

In [14]:
class Timer(object):
    def __enter__(self):
        self.t0 = time.time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        print('[time spent: {time:.2f}s]'.format(time = time.time() - self.t0))

In [15]:
B = 64       # Batch size
F = 10       # Number of observed snappshots
D = 1024     # Dimension of Koopman embedding
G = 10       # Number of successively incoming snapshots

Z = torch.randn(B, F, D)                # Observed time points
incoming_list = torch.randn(B, G, D)    # Successively incoming time points

In [16]:
# Algorithm 1
with Timer():
    pred1 = operator_adaptation(Z, incoming_list)

[time spent: 5.59s]


In [17]:
# Algorithm 2
with Timer():
    pred2 = operator_adaptation_accelerate(Z, incoming_list)

[time spent: 2.68s]


In [18]:
# equivalent results
assert torch.norm(pred1-pred2) < 1e-3

Notably, we propsoe the accelerated version of operator adaptation **involving length mismatches and long-term dynamics forecast scenarios**. In the realm of the Koopman method, the long-term applicability are exceptional.